In [13]:
# Add adjustments 

import pandas as pd
import numpy as np
import random
import statistics
from sklearn.linear_model import LinearRegression
import time
from sklearn import linear_model
import statsmodels.api as sm
import statistics as stats

from Metrics import Metrics

data = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Final_data.xlsx")
# Izberi nakjlučni vzorec velikosti 300

# Clean sample

data_test = data[data.Train_test_identificator == "test"]
data_train = data[data.Train_test_identificator == "train"]



def add_adjustments(modeling_results):
    
    
    # Univariatna linearna regresija
    
    # Odfiltriraj robne cene

    min_cena = data_train["cena"].quantile(0.1)
    max_cena = data_train["cena"].quantile(0.9)
    data_train_for_univariate_linear_regression = data_train[(data_train.cena > min_cena) &  (data_train.cena < max_cena) ]


    # Fit starost

    data_train_for_multivariate_linear_regression_starost = data_train_for_univariate_linear_regression.dropna(subset=['starost'])

    X_starost = data_train_for_multivariate_linear_regression_starost[['starost']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y_starost = data_train_for_multivariate_linear_regression_starost['cena']

    regr_starost  = linear_model.LinearRegression()
    regr_starost.fit(X_starost, Y_starost)

    # Fit prevozeni_km

    data_train_for_multivariate_linear_regression_prevozeni_km = data_train_for_univariate_linear_regression.dropna(subset=['prevozeni_km'])

    X_prevozeni_km = data_train_for_multivariate_linear_regression_prevozeni_km[['prevozeni_km']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y_prevozeni_km = data_train_for_multivariate_linear_regression_prevozeni_km['cena']

    regr_prevozeni_km  = linear_model.LinearRegression()
    regr_prevozeni_km.fit(X_prevozeni_km, Y_prevozeni_km)

    # Fit ccm_class

    data_train_for_multivariate_linear_regression_ccm_class = data_train_for_univariate_linear_regression.dropna(subset=['ccm_class'])

    X_ccm_class = data_train_for_multivariate_linear_regression_ccm_class[['ccm_class']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y_ccm_class = data_train_for_multivariate_linear_regression_ccm_class['cena']

    regr_ccm_class  = linear_model.LinearRegression()
    regr_ccm_class.fit(X_ccm_class, Y_ccm_class)


    adjusted_ocene = []
    adjusted_prices_vse = []

    for i in range(0,len(modeling_results.index)):
        string = modeling_results["IDS_of_top_k_matches"].values[i][1:-1]
        words = []
        word = ""
        for s in string:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words.append(int(copy_of_word))

        adjusted_prices = []

        for w in words:
            Comparable_data = data_train[data_train.ID == w]
            Main_data = data_test[data_test.ID == modeling_results["ID"].values[i]]

            # Code above is irrelevant but needed

            # Now we have the main and the comparable cars
            # We predict the values using the linear multivariate regression

            starost_main = Main_data["starost"].values[0]
            prevozeni_km_main = Main_data["prevozeni_km"].values[0]
            ccm_class_main = Main_data["ccm_class"].values[0]


            starost_comp = Comparable_data["starost"].values[0]
            prevozeni_km_comp = Comparable_data["prevozeni_km"].values[0]
            ccm_class_comp =  Comparable_data["ccm_class"].values[0]


            # Ker imamo nekatere mankajoče vrednosti pri comparablih nemorremo pri vseh spremeniti ocen


            try:
                 # Prediction for main startost

                prediction_main = regr_starost.predict([[starost_main]])[0]

                # Prediction comp starost

                prediction_comp = regr_starost.predict([[starost_comp]])[0]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * Comparable_data["cena"].values[0]

            except ValueError:
                # Če regresije ne moremo opraviti za adjusted price vzamemo kar orignalno ceno
                adjusted_price = Comparable_data["cena"].values[0]

            try:
                 # Prediction for main prevozeni_km 

                prediction_main = regr_prevozeni_km.predict([[prevozeni_km_main]])[0]

                # Prediction comp prevozeni km

                prediction_comp = regr_prevozeni_km.predict([[prevozeni_km_comp]])[0]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * adjusted_price

            except ValueError:
                # Če regresije ne moremo opraviti za adjusted price vzamemo kar prejšnjo oceno
                adjusted_price = adjusted_price


            try:
                 # Prediction for main ccm class

                prediction_main = regr_prevozeni_km.predict([[ccm_class_main]])[0]

                # Prediction comp ccm c las

                prediction_comp = regr_prevozeni_km.predict([[ccm_class_comp]])[0]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * adjusted_price

            except ValueError:
                # Če regresije ne moremo opraviti za adjusted price vzamemo kar prejšnjo oceno
                adjusted_price = adjusted_price


            adjusted_prices.append(adjusted_price)


        adjusted_prices = [float(i) for i in adjusted_prices]
        adjusted_ocena = stats.mean(adjusted_prices)

        adjusted_prices_vse.append(adjusted_prices)
        adjusted_ocene.append(adjusted_ocena)

    modeling_results["Adjusted_prices_z_univariatno_regresijo"] = adjusted_prices_vse
    modeling_results["ocena_z_univariatno_regresijo"] = adjusted_ocene
    

    
    
    # Multivariatna linearna regresija

    # Odfiltriraj robne cene
    min_cena = data_train["cena"].quantile(0.1)
    max_cena = data_train["cena"].quantile(0.9)
    data_train_for_multivariate_linear_regression = data_train[(data_train.cena > min_cena) &  (data_train.cena < max_cena) ]

    # Drop missing values
    data_train_for_multivariate_linear_regression = data_train_for_multivariate_linear_regression.dropna(subset=['starost','prevozeni_km', "ccm_class"])

    # Fit multivariate linear regression model

    X = data_train_for_multivariate_linear_regression[['starost','prevozeni_km', "ccm_class"]] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y = data_train_for_multivariate_linear_regression['cena']

    # with sklearn
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)


    adjusted_ocene = []
    adjusted_prices_vse = []

    for i in range(0,len(modeling_results.index)):
        string = modeling_results["IDS_of_top_k_matches"].values[i][1:-1]
        words = []
        word = ""
        for s in string:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words.append(int(copy_of_word))

        adjusted_prices = []

        for w in words:
            Comparable_data = data_train[data_train.ID == w]
            Main_data = data_test[data_test.ID == modeling_results["ID"].values[i]]

            # Code above is irrelevant but needed

            # Now we have the main and the comparable cars
            # We predict the values using the linear multivariate regression

            starost_main = Main_data["starost"].values[0]
            prevozeni_km_main = Main_data["prevozeni_km"].values[0]
            ccm_class_main = Main_data["ccm_class"].values[0]


            starost_comp = Comparable_data["starost"].values[0]
            prevozeni_km_comp = Comparable_data["prevozeni_km"].values[0]
            ccm_class_comp =  Comparable_data["ccm_class"].values[0]


            # Ker imamo nekatere mankajoče vrednosti pri comparablih nemorremo pri vseh spremeniti ocene
            # Prediction for main
            
            try:
                prediction_main = regr.predict([[starost_main,prevozeni_km_main,ccm_class_main]])[0]

                # Prediction comp

                prediction_comp = regr.predict([[starost_comp,prevozeni_km_comp,ccm_class_comp]])[0]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * Comparable_data["cena"].values[0]

                adjusted_prices.append(adjusted_price)
            except ValueError:
                # Če regresije ne moremo opraviti za adjusted price vzamemo kar orignalno ceno
                adjusted_price = Comparable_data["cena"].values[0]
                adjusted_prices.append(adjusted_price)


        adjusted_prices = [float(i) for i in adjusted_prices]
        adjusted_ocena = stats.mean(adjusted_prices)

        adjusted_prices_vse.append(adjusted_prices)
        adjusted_ocene.append(adjusted_ocena)
        
        

    modeling_results["Adjusted_prices_z_multivariatno_regresijo"] = list(adjusted_prices_vse)
    modeling_results["ocena_z_multivariatno_regresijo"] = list(adjusted_ocene)
    
    # Multivariatna regresija z vključeno razvrstitev znamke
    

    # Odfiltriraj robne cene
    min_cena = data_train["cena"].quantile(0.1)
    max_cena = data_train["cena"].quantile(0.9)
    data_train_for_multivariate_linear_regression = data_train[(data_train.cena > min_cena) &  (data_train.cena < max_cena) ]

    # Drop missing values
    data_train_for_multivariate_linear_regression = data_train_for_multivariate_linear_regression.dropna(subset=['starost','prevozeni_km', "ccm_class"])

    # Fit multivariate linear regression model

    X = data_train_for_multivariate_linear_regression[['starost','prevozeni_km', "razvrstitev_povprecne_cene_po_tipu"]] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y = data_train_for_multivariate_linear_regression['cena']

    # with sklearn
    regr2 = linear_model.LinearRegression()
    regr2.fit(X, Y)


    adjusted_ocene = []
    adjusted_prices_vse = []

    for i in range(0,len(modeling_results.index)):
        string = modeling_results["IDS_of_top_k_matches"].values[i][1:-1]
        words = []
        word = ""
        for s in string:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words.append(int(copy_of_word))

        adjusted_prices = []

        for w in words:
            Comparable_data = data_train[data_train.ID == w]
            Main_data = data_test[data_test.ID == modeling_results["ID"].values[i]]

            # Code above is irrelevant but needed

            # Now we have the main and the comparable cars
            # We predict the values using the linear multivariate regression

            starost_main = Main_data["starost"].values[0]
            prevozeni_km_main = Main_data["prevozeni_km"].values[0]
            ccm_class_main = Main_data["ccm_class"].values[0]
            ranking_main = Main_data["razvrstitev_povprecne_cene_po_tipu"].values[0]


            starost_comp = Comparable_data["starost"].values[0]
            prevozeni_km_comp = Comparable_data["prevozeni_km"].values[0]
            ccm_class_comp =  Comparable_data["ccm_class"].values[0]
            ranking_com = Comparable_data["razvrstitev_povprecne_cene_po_tipu"].values[0]


            # Ker imamo nekatere mankajoče vrednosti pri comparablih nemorremo pri vseh spremeniti ocene
            # Prediction for main
            
            try:
                prediction_main = regr2.predict([[starost_main,
                                                  prevozeni_km_main,
                                                 ranking_main]])[0]

                # Prediction comp

                prediction_comp = regr2.predict([[starost_comp,
                                                  prevozeni_km_comp,
                                                 ranking_com]])[0]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * Comparable_data["cena"].values[0]

                adjusted_prices.append(adjusted_price)
            except ValueError:
                # Če regresije ne moremo opraviti za adjusted price vzamemo kar orignalno ceno
                adjusted_price = Comparable_data["cena"].values[0]
                adjusted_prices.append(adjusted_price)


        adjusted_prices = [float(i) for i in adjusted_prices]
        adjusted_ocena = stats.mean(adjusted_prices)

        adjusted_prices_vse.append(adjusted_prices)
        adjusted_ocene.append(adjusted_ocena)
        
        

    modeling_results["Adjusted_prices_z_multivariatno_regresijo_v2"] = adjusted_prices_vse
    modeling_results["ocena_z_multivariatno_regresijo_v2"] = adjusted_ocene
    
    # Konec multivariatne 
    
    # logarithmic regression
    data_prevozeni_km = data_train.dropna(subset = ["prevozeni_km"])

    y = data_prevozeni_km["cena"].values
    x = data_prevozeni_km["prevozeni_km"].values


    fit_prevozeni_km = np.polyfit(np.log(x), y, 1)

    # Starost 

    # Starost zamakni za 1

    data_starost = data_train.dropna(subset = ["starost"])

    y = data_starost["cena"].values
    x = np.array(data_starost["starost"] + 1)


    fit_starost = np.polyfit(np.log(x), y, 1)


    # Starost 

    # Starost zamakni za 1

    data_ccm = data_train.dropna(subset = ["ccm_class"])

    y = data_ccm["cena"].values
    x = np.array(data_ccm["ccm_class"] + 1)

    fit_ccm = np.polyfit(np.log(x), y, 1)
        
    adjusted_price_all = []
    adjusted_ocene_log_out_mean = []
    
    for i in range(0,len(modeling_results.index)):
        string = modeling_results["IDS_of_top_k_matches"].values[i][1:-1]
        words = []
        word = ""
        for s in string:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words.append(int(copy_of_word))

        adjusted_price_log = []

        for w in words:
            Comparable_data = data_train[data_train.ID == w]
            Main_data = data_test[data_test.ID == modeling_results["ID"].values[i]]

            # Code above is irrelevant but needed

            # Now we have the main and the comparable cars
            # We predict the values using linear multivariate regression

            starost_main = Main_data["starost"].values[0]
            prevozeni_km_main = Main_data["prevozeni_km"].values[0]
            ccm_class_main = Main_data["ccm_class"].values[0]


            starost_comp = Comparable_data["starost"].values[0]
            prevozeni_km_comp = Comparable_data["prevozeni_km"].values[0]
            ccm_class_comp =  Comparable_data["ccm_class"].values[0]
            
            # Starost
            # pri starosti moraš 1 prišteti, ker je log(0) problem.
            try:
                prediction_main = fit_ccm[0] * np.log(starost_main+1) + fit_ccm[1]

                # Prediction comp

                prediction_comp =  fit_ccm[0] * np.log(starost_comp+1) + fit_ccm[1]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * Comparable_data["cena"].values[0]
                
            except ValueError:
                 adjusted_price =  adjusted_price
                    
            
            # Prevozeni km
            
            try:
                prediction_main = fit_ccm[0] * np.log(prevozeni_km_main) + fit_ccm[1]

                # Prediction comp

                prediction_comp =  fit_ccm[0] * np.log(prevozeni_km_comp) + fit_ccm[1]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * adjusted_price
                
            except ValueError:
                 adjusted_price =  adjusted_price
                    
                    
            # ccm
            try:
                prediction_main = fit_ccm[0] * np.log(ccm_class_main) + fit_ccm[1]

                # Prediction comp

                prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]

                # Adjustment is going to be equal to the relative difference between the predictions

                adjustment = (prediction_main-prediction_comp) / prediction_comp

                adjusted_price = (1 +adjustment) * adjusted_price
                
            except ValueError:
                 adjusted_price =  adjusted_price
            
            adjusted_price_log.append(adjusted_price)
    

        adjusted_price_log = [float(i) for i in adjusted_price_log]
        adjusted_ocena_log_mean = stats.mean(adjusted_price_log)

        adjusted_price_all.append(adjusted_price_log)
        adjusted_ocene_log_out_mean.append(adjusted_ocena_log_mean)
    
    
    modeling_results["Adjusted_prices_z_logaritmicno_regresijo"] = adjusted_price_all
    modeling_results["ocena_z_logaritmicno_regresijo"] = adjusted_ocene_log_out_mean       
    return modeling_results

In [14]:
data = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Final_data.xlsx")
# Izberi nakjlučni vzorec velikosti 300

# Clean sample

data_test = data[data.Train_test_identificator == "test"]
data_train = data[data.Train_test_identificator == "train"]



import pandas as pd
import numpy as np
import random
import statistics
from sklearn.linear_model import LinearRegression
import time

def linear(x):
    return x

def square(x):
    return x**2

def sqrt(x):
    return x**0.5

def normalize(f,vector):
    vector_f = [f(v) for v in vector]
    sum_vector_f = sum(vector_f)
    return [v / sum_vector_f for v in vector_f]



import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
import statistics as stats


def add_weights(Model_results_with_adj):
    
    ocena_kvadratna_normalizacija_list = []
    ocena_linearna_normalizacija_list = []
    ocena_korenska_normalizacija_list = []

    Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija_list = []
    Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija_list = []
    Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija_list = []

    Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list = []
    Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list = []
    Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list = []
    
    Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list_v2 = []
    Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list_v2 = []
    Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list_v2 = []

    Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija_list = []
    Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija_list = []
    Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija_list = []



    for i in range(0,len(Model_results_with_adj.index)):
        string = Model_results_with_adj["Mere_podobnosti"].values[i][1:-1]
        words = []
        word = ""
        for s in string:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words.append(int(copy_of_word))
        
        # Pridobi id-je comparablov
        
        string2 = Model_results_with_adj["IDS_of_top_k_matches"].values[i][1:-1]
        words2 = []
        word = ""
        for s in string2:
            if s == " ":
                try:
                    copy_of_word = int(word)
                    words2.append(int(copy_of_word))
                    word = ""
                except:
                    a=1
            else:
                word = word + s
                int(word)

        copy_of_word = int(word)
        words2.append(int(copy_of_word))
        
        cene_comparable = []
        
        for Id in words2:
            d = data_train[data_train.ID == Id]
            cene_comparable.append(d["cena"].values[0])
        # izračunaj uteži glede na različne funkcije

        vector_podobnosti = list(words)

        kvadratna_normalizacija =  np.array(normalize(square,vector_podobnosti))
        linearna_normalizacija =  np.array(normalize(linear,vector_podobnosti))
        korenska_normalizacija =  np.array(normalize(sqrt,vector_podobnosti))

        list_of_prices = ["ocena", 
                      "Adjusted_prices_z_univariatno_regresijo", 
                      "Adjusted_prices_z_multivariatno_regresijo",
                      "Adjusted_prices_z_logaritmicno_regresijo"]

        ocena_kvadratna_normalizacija = kvadratna_normalizacija * np.array(cene_comparable)
        ocena_linearna_normalizacija = linearna_normalizacija * np.array(cene_comparable)
        ocena_korenska_normalizacija = korenska_normalizacija * np.array(cene_comparable)

        Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija = kvadratna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo"].values[i])
        Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija = linearna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo"].values[i])
        Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija = korenska_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo"].values[i])    

        Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija = kvadratna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo"].values[i])
        Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija = linearna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo"].values[i])
        Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija = korenska_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo"].values[i])

        
        Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_v2 = kvadratna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_v2"].values[i])
        Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_v2 = linearna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_v2"].values[i])
        Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_v2 = korenska_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_v2"].values[i])

        Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija = kvadratna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo"].values[i])
        Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija = linearna_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo"].values[i])
        Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija = korenska_normalizacija * np.array(Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo"].values[i])

        # Zapolni sezname 
        ocena_kvadratna_normalizacija_list.append(sum(ocena_kvadratna_normalizacija))
        ocena_linearna_normalizacija_list.append(sum(ocena_linearna_normalizacija))
        ocena_korenska_normalizacija_list.append(sum(ocena_korenska_normalizacija))

        Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija_list.append(sum(Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija))
        Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija_list.append(sum(Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija))
        Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija_list.append(sum(Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija))

        Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list.append(sum(Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija))
        Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list.append(sum(Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija)) 
        Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list.append(sum(Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija)) 

        Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list_v2.append(sum(Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_v2))
        Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list_v2.append(sum(Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_v2)) 
        Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list_v2.append(sum(Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_v2)) 

        Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija_list.append(sum(Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija))
        Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija_list.append(sum(Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija)) 
        Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija_list.append(sum(Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija))

    Model_results_with_adj["ocena_kvadratna_normalizacija"] = ocena_kvadratna_normalizacija_list
    Model_results_with_adj["ocena_linearna_normalizacija" ] = ocena_linearna_normalizacija_list
    Model_results_with_adj["ocena_korenska_normalizacija"] = ocena_korenska_normalizacija_list
    

    Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija"] = Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija"] = Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija"] = Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija_list

    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija"] = Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija"] = Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija"] = Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list

    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_v2"] = Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_list_v2
    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_v2"] = Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_list_v2
    Model_results_with_adj["Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_v2"] = Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_list_v2

    Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija"] = Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija"] = Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija_list
    Model_results_with_adj["Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija"] = Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija_list
        
    return Model_results_with_adj


def get_weights_results(Model_results_with_adj):
    parametri = Model_results_with_adj["Parameters"].values[0]
    Model_results_with_adj_2 = add_weights(Model_results_with_adj)
    name_of_run = "Run 1"
    columns_for_metrics = ["ocena",
                    "ocena_z_univariatno_regresijo",
                    "ocena_z_multivariatno_regresijo",
                    "ocena_z_multivariatno_regresijo_v2",
                    "ocena_z_logaritmicno_regresijo",
                    'ocena_kvadratna_normalizacija',
                    'ocena_linearna_normalizacija', 
                    'ocena_korenska_normalizacija',
                    'Adjusted_prices_z_univariatno_regresijo_kvadratna_normalizacija',
                    'Adjusted_prices_z_univariatno_regresijo_linearna_normalizacija',
                    'Adjusted_prices_z_univariatno_regresijo_korenska_normalizacija',
                    'Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija',
                    'Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija',
                    'Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija',
                    'Adjusted_prices_z_multivariatno_regresijo_kvadratna_normalizacija_v2',
                    'Adjusted_prices_z_multivariatno_regresijo_linearna_normalizacija_v2',
                    'Adjusted_prices_z_multivariatno_regresijo_korenska_normalizacija_v2',                           
                    'Adjusted_prices_z_logaritmicno_regresijo_kvadratna_normalizacija',
                    'Adjusted_prices_z_logaritmicno_regresijo_linearna_normalizacija',
                    'Adjusted_prices_z_logaritmicno_regresijo_korenska_normalizacija']

    for i in range(0,len(columns_for_metrics)):
        if i == 0:
            df = Metrics(add_weights(Model_results_with_adj), columns_for_metrics[i] , columns_for_metrics[i] ,parametri)
        else:
            df2 = Metrics(add_weights(Model_results_with_adj), columns_for_metrics[i] , columns_for_metrics[i] ,parametri)

            df = df.append(pd.DataFrame(df2))

    return df
    

In [34]:
for i in range(1,41):
    print(i)
    modeling_results = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Modeling results\\Archive\\New_x0_" + str(i) + ".xlsx")
    if i == 1:
        Model_results_with_adj = add_adjustments(modeling_results)
        df_out = pd.DataFrame(get_weights_results(Model_results_with_adj))
    else:
        Model_results_with_adj = add_adjustments(modeling_results)
        df_out = df_out.append(get_weights_results(Model_results_with_adj))
    
df_out.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Modeling results\\new_x0_calculated_results.xlsx")

1


<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp
<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp


2


<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp
<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp


3


...

37


<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp
<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp


38
39


<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp
<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp
<ipython-input-22-f2d8abc5f5d5>:503: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-22-f2d8abc5f5d5>:507: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp


40


In [23]:
mypath = "C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\"


from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
k = 1

for i in onlyfiles:
    file_name = mypath + i
    print(i)
    modeling_results = pd.read_excel(file_name)
    modeling_results = modeling_results[modeling_results["cena"] <= 5000]
    if k == 1:
        Model_results_with_adj = add_adjustments(modeling_results)
        df_out = pd.DataFrame(get_weights_results(Model_results_with_adj))
    else:
        Model_results_with_adj = add_adjustments(modeling_results)
        df_out = df_out.append(get_weights_results(Model_results_with_adj))
    k = 2
df_out.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Modeling results\\Final metrics cena pod 5000.xlsx")

Final estimations.xlsx


<ipython-input-13-870f24b297b0>:510: RuntimeWarning: divide by zero encountered in log
  prediction_comp =  fit_ccm[0] * np.log(ccm_class_comp) + fit_ccm[1]
<ipython-input-13-870f24b297b0>:514: RuntimeWarning: invalid value encountered in double_scalars
  adjustment = (prediction_main-prediction_comp) / prediction_comp


In [20]:
modeling_results

,Unnamed: 0,Sample,Count,Error within 5 %,Error within 10 %,Error within 15 %,Error within 20 %,Error within 25 %,Error within 30 %,Error within 35 %,...,Error within 45 %,Error within 50 %,Median Absolute Error,Mean Absolute Error,Max Absolute Error,Median Relative_error,Median Absolute Relative Error,Mean Abosolute Relative Error,Max Absolute Relative Error,Parameters_used
0,0,ocena,1000,0.215,0.406,0.547,0.662,0.743,0.805,0.845,...,0.903,0.923,1197.500000,2081.558500,76377.500000,0.009899,0.131522,0.215815,8.442381,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
1,0,ocena_z_univariatno_regresijo,1000,0.233,0.413,0.570,0.690,0.761,0.821,0.856,...,0.910,0.926,1114.610643,1974.870510,75909.033221,0.008058,0.123253,0.213868,7.318827,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
2,0,ocena_z_multivariatno_regresijo,1000,0.202,0.384,0.539,0.665,0.745,0.817,0.857,...,0.903,0.913,1208.962801,2032.867634,75562.373425,0.000909,0.135145,0.225871,6.348981,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
3,0,ocena_z_multivariatno_regresijo_v2,1000,0.231,0.400,0.553,0.654,0.721,0.775,0.814,...,0.859,0.874,1318.364754,2401.978640,145302.732772,0.005187,0.131007,0.390365,69.191778,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
4,0,ocena_z_logaritmicno_regresijo,1000,0.196,0.370,0.502,0.605,0.698,0.765,0.818,...,0.885,0.903,5729.120735,NaN,76191.083521,NaN,NaN,NaN,8.564453,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
5,0,ocena_kvadratna_normalizacija,1000,0.198,0.374,0.521,0.647,0.731,0.787,0.832,...,0.885,0.914,1294.931660,2218.928406,68325.478795,0.006776,0.142296,0.232713,8.080161,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
6,0,ocena_linearna_normalizacija,1000,0.218,0.380,0.555,0.655,0.734,0.799,0.846,...,0.893,0.921,1261.498728,2133.465494,72852.952234,0.003530,0.134774,0.223742,8.015357,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
7,0,ocena_korenska_normalizacija,1000,0.213,0.401,0.551,0.654,0.739,0.806,0.846,...,0.901,0.920,1225.590678,2092.304678,74825.755171,0.006115,0.130360,0.218837,8.121334,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
8,0,Adjusted_prices_z_univariatno_regresijo_kvadra...,1000,0.222,0.393,0.557,0.661,0.745,0.803,0.841,...,0.896,0.907,1218.225528,2094.564971,67805.611296,-0.002450,0.132465,0.228297,6.843509,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
9,0,Adjusted_prices_z_univariatno_regresijo_linear...,1000,0.225,0.403,0.571,0.679,0.757,0.805,0.846,...,0.901,0.916,1192.202893,2020.239635,72310.187262,-0.000573,0.127642,0.221587,6.770704,"[466.08949, 66.312822, 8.032453, 24.750775, 0...."
